# 68-持续集成与持续部署

## 📚 用途说明

**学习目标**：
- 掌握持续集成（CI）的核心概念和流程
- 学会持续部署（CD）的策略和实现
- 理解DevOps文化和自动化工具链
- 能够建立完整的CI/CD流水线

**前置要求**：
- 已完成测试驱动开发与质量保证学习
- 熟练掌握版本控制和协作开发
- 了解软件构建和部署流程
- 具备基本的自动化意识

**与LangChain关联**：
- 支持LangChain项目的自动化构建和部署
- 为AI模型训练和部署提供CI/CD方案
- 保障LangChain服务的持续交付能力
- 实现AI系统的自动化运维

---

## 🔢 知识点覆盖

### 8.4 持续集成与持续部署 [⭐⭐进阶]
**知识点说明**：持续集成与持续部署是现代软件工程的核心实践，为LangChain的AI项目提供高效的自动化交付能力。

**学习要求**：
- 掌握CI/CD的核心概念和最佳实践
- 学会构建自动化流水线
- 理解部署策略和环境管理
- 能够建立完整的DevOps流程

**案例要求**：
- 实现CI/CD流水线框架
- 构建自动化构建和测试系统
- 开发部署策略管理器
- 验证点：能独立建立完整的CI/CD体系

In [ ]:
print("🚀 持续集成与持续部署:")
print("=" * 50)

# 导入必要的库
import os
import sys
import json
import datetime
import time
import subprocess
import threading
import queue
import hashlib
import shutil
import tempfile
import pathlib
import typing
import dataclasses
import uuid
import yaml
from typing import List, Dict, Any, Optional, Union, Tuple, Callable
from dataclasses import dataclass, field, asdict
from enum import Enum
from abc import ABC, abstractmethod
from pathlib import Path

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 持续集成（CI）系统
print(f"📝 1. 持续集成（CI）系统:")

# 1.1 CI核心概念
print(f"\n   🎯 1.1 CI核心概念:")

class BuildStatus(Enum):
    """构建状态"""
    PENDING = "pending"  # 等待中
    RUNNING = "running"  # 运行中
    SUCCESS = "success"  # 成功
    FAILED = "failed"  # 失败
    CANCELLED = "cancelled"  # 取消
    TIMEOUT = "timeout"  # 超时

class TriggerType(Enum):
    """触发类型"""
    COMMIT = "commit"  # 代码提交
    SCHEDULE = "schedule"  # 定时触发
    MANUAL = "manual"  # 手动触发
    WEBHOOK = "webhook"  # Webhook触发
    PR = "pull_request"  # Pull Request

@dataclass
class BuildStep:
    """构建步骤"""
    name: str
    command: str
    timeout: int = 300
    retry_count: int = 0
    continue_on_error: bool = False
    working_directory: str = ""
    environment: Dict[str, str] = field(default_factory=dict)
    dependencies: List[str] = field(default_factory=list)
    
    def __post_init__(self):
        self.status = BuildStatus.PENDING
        self.start_time: Optional[datetime.datetime] = None
        self.end_time: Optional[datetime.datetime] = None
        self.output: str = ""
        self.error_message: Optional[str] = None

@dataclass
class BuildJob:
    """构建任务"""
    id: str
    name: str
    repository: str
    branch: str
    commit_hash: str
    trigger_type: TriggerType
    triggered_by: str
    steps: List[BuildStep]
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    
    def __post_init__(self):
        self.status = BuildStatus.PENDING
        self.start_time: Optional[datetime.datetime] = None
        self.end_time: Optional[datetime.datetime] = None
        self.artifacts: List[str] = []
        self.environment: Dict[str, str] = {}

@dataclass
class BuildResult:
    """构建结果"""
    job: BuildJob
    status: BuildStatus
    execution_time: float
    step_results: List[Dict[str, Any]] = field(default_factory=list)
    artifacts: List[str] = field(default_factory=list)
    test_results: Optional[Dict[str, Any]] = None
    coverage_data: Optional[Dict[str, Any]] = None
    error_message: Optional[str] = None

class ContinuousIntegrationServer:
    """持续集成服务器"""
    
    def __init__(self):
        self.build_queue: queue.Queue = queue.Queue()
        self.active_builds: Dict[str, BuildJob] = {}
        self.build_history: List[BuildResult] = []
        self.build_workers: List[threading.Thread] = []
        self.max_workers = 3
        self.is_running = False
        self.build_configs: Dict[str, Dict[str, Any]] = {}
    
    def start(self):
        """启动CI服务器"""
        self.is_running = True
        
        # 启动工作线程
        for i in range(self.max_workers):
            worker = threading.Thread(target=self._build_worker, name=f"BuildWorker-{i}")
            worker.daemon = True
            worker.start()
            self.build_workers.append(worker)
        
        print(f"CI服务器启动，运行 {self.max_workers} 个构建工作线程")
    
    def stop(self):
        """停止CI服务器"""
        self.is_running = False
        print("CI服务器停止")
    
    def submit_build(self, job: BuildJob) -> bool:
        """提交构建任务"""
        try:
            self.build_queue.put(job)
            print(f"构建任务提交成功: {job.id} - {job.name}")
            return True
        except Exception as e:
            print(f"构建任务提交失败: {e}")
            return False
    
    def _build_worker(self):
        """构建工作线程"""
        while self.is_running:
            try:
                # 获取构建任务（阻塞等待）
                job = self.build_queue.get(timeout=1)
                
                # 执行构建
                result = self._execute_build(job)
                
                # 记录结果
                self.build_history.append(result)
                
                # 清理活跃构建
                if job.id in self.active_builds:
                    del self.active_builds[job.id]
                
                print(f"构建完成: {job.id} - {result.status.value}")
                
            except queue.Empty:
                continue
            except Exception as e:
                print(f"构建工作线程错误: {e}")
    
    def _execute_build(self, job: BuildJob) -> BuildResult:
        """执行构建"""
        job.status = BuildStatus.RUNNING
        job.start_time = datetime.datetime.utcnow()
        self.active_builds[job.id] = job
        
        start_time = time.time()
        step_results = []
        artifacts = []
        
        try:
            # 创建构建环境
            build_env = self._create_build_environment(job)
            
            # 执行构建步骤
            for step in job.steps:
                step_result = self._execute_step(step, build_env)
                step_results.append(step_result)
                
                # 如果步骤失败且不允许继续，停止构建
                if step_result["status"] == BuildStatus.FAILED and not step.continue_on_error:
                    break
            
            # 确定最终状态
            failed_steps = [r for r in step_results if r["status"] == BuildStatus.FAILED]
            if failed_steps:
                status = BuildStatus.FAILED
                error_message = failed_steps[0]["error_message"]
            else:
                status = BuildStatus.SUCCESS
                error_message = None
            
            # 收集构建产物
            artifacts = self._collect_artifacts(job, build_env)
            
        except Exception as e:
            status = BuildStatus.FAILED
            error_message = str(e)
        
        finally:
            # 清理构建环境
            self._cleanup_build_environment(build_env)
            
            execution_time = time.time() - start_time
            job.end_time = datetime.datetime.utcnow()
            job.status = status
        
        return BuildResult(
            job=job,
            status=status,
            execution_time=execution_time,
            step_results=step_results,
            artifacts=artifacts,
            error_message=error_message
        )
    
    def _create_build_environment(self, job: BuildJob) -> Dict[str, Any]:
        """创建构建环境"""
        build_dir = tempfile.mkdtemp(prefix=f"build_{job.id}_")
        
        env = {
            "build_directory": build_dir,
            "job_id": job.id,
            "branch": job.branch,
            "commit_hash": job.commit_hash,
            "repository": job.repository
        }
        
        print(f"创建构建环境: {build_dir}")
        return env
    
    def _execute_step(self, step: BuildStep, build_env: Dict[str, Any]) -> Dict[str, Any]:
        """执行构建步骤"""
        step.status = BuildStatus.RUNNING
        step.start_time = datetime.datetime.utcnow()
        
        start_time = time.time()
        output = ""
        error_message = None
        
        try:
            # 模拟执行命令
            print(f"执行步骤: {step.name} - {step.command}")
            
            # 模拟命令执行时间
            time.sleep(0.1)
            
            # 模拟命令输出
            output = f"执行命令: {step.command}\n输出: Step completed successfully"
            
            # 模拟随机失败（10%概率）
            import random
            if random.random() < 0.1:
                raise RuntimeError(f"Step failed: {step.name}")
            
            step.status = BuildStatus.SUCCESS
            status = BuildStatus.SUCCESS
            
        except Exception as e:
            step.status = BuildStatus.FAILED
            status = BuildStatus.FAILED
            error_message = str(e)
            output = f"错误: {error_message}"
        
        finally:
            execution_time = time.time() - start_time
            step.end_time = datetime.datetime.utcnow()
            step.output = output
            step.error_message = error_message
        
        return {
            "step_name": step.name,
            "status": status,
            "execution_time": execution_time,
            "output": output,
            "error_message": error_message
        }
    
    def _collect_artifacts(self, job: BuildJob, build_env: Dict[str, Any]) -> List[str]:
        """收集构建产物"""
        # 模拟收集构建产物
        artifacts = [
            f"{job.name}_build.log",
            f"{job.name}_test_report.html",
            f"{job.name}_coverage.xml"
        ]
        
        print(f"收集构建产物: {len(artifacts)} 个文件")
        return artifacts
    
    def _cleanup_build_environment(self, build_env: Dict[str, Any]):
        """清理构建环境"""
        build_dir = build_env.get("build_directory")
        if build_dir and os.path.exists(build_dir):
            try:
                shutil.rmtree(build_dir)
                print(f"清理构建环境: {build_dir}")
            except Exception as e:
                print(f"清理构建环境失败: {e}")
    
    def get_build_status(self, job_id: str) -> Optional[Dict[str, Any]]:
        """获取构建状态"""
        # 检查活跃构建
        if job_id in self.active_builds:
            job = self.active_builds[job_id]
            return {
                "job_id": job_id,
                "status": job.status,
                "start_time": job.start_time.isoformat() if job.start_time else None,
                "progress": "running"
            }
        
        # 检查历史构建
        for result in self.build_history:
            if result.job.id == job_id:
                return {
                    "job_id": job_id,
                    "status": result.status,
                    "start_time": result.job.start_time.isoformat() if result.job.start_time else None,
                    "end_time": result.job.end_time.isoformat() if result.job.end_time else None,
                    "execution_time": result.execution_time,
                    "progress": "completed"
                }
        
        return None
    
    def get_build_statistics(self) -> Dict[str, Any]:
        """获取构建统计"""
        total_builds = len(self.build_history)
        successful_builds = len([r for r in self.build_history if r.status == BuildStatus.SUCCESS])
        failed_builds = len([r for r in self.build_history if r.status == BuildStatus.FAILED])
        
        total_time = sum(r.execution_time for r in self.build_history)
        
        return {
            "total_builds": total_builds,
            "successful_builds": successful_builds,
            "failed_builds": failed_builds,
            "success_rate": successful_builds / total_builds if total_builds > 0 else 0,
            "active_builds": len(self.active_builds),
            "queued_builds": self.build_queue.qsize(),
            "average_build_time": total_time / total_builds if total_builds > 0 else 0
        }

# 1.2 CI配置管理
print(f"\n   ⚙️ 1.2 CI配置管理:")

@dataclass
class CIConfiguration:
    """CI配置"""
    name: str
    repository: str
    branches: List[str]
    triggers: List[TriggerType]
    environment: Dict[str, str]
    build_steps: List[BuildStep]
    artifacts: List[str]
    notifications: Dict[str, Any]
    
    def to_yaml(self) -> str:
        """转换为YAML格式"""
        config_dict = {
            "name": self.name,
            "repository": self.repository,
            "branches": self.branches,
            "triggers": [t.value for t in self.triggers],
            "environment": self.environment,
            "build_steps": [
                {
                    "name": step.name,
                    "command": step.command,
                    "timeout": step.timeout,
                    "retry_count": step.retry_count,
                    "continue_on_error": step.continue_on_error,
                    "working_directory": step.working_directory,
                    "environment": step.environment
                }
                for step in self.build_steps
            ],
            "artifacts": self.artifacts,
            "notifications": self.notifications
        }
        
        return yaml.dump(config_dict, default_flow_style=False, allow_unicode=True)

class CIConfigurationManager:
    """CI配置管理器"""
    
    def __init__(self):
        self.configurations: Dict[str, CIConfiguration] = {}
        self.templates: Dict[str, CIConfiguration] = {}
        self._load_default_templates()
    
    def _load_default_templates(self):
        """加载默认模板"""
        # Python项目模板
        python_template = CIConfiguration(
            name="Python Project",
            repository="",
            branches=["main", "develop"],
            triggers=[TriggerType.COMMIT, TriggerType.PR],
            environment={
                "PYTHON_VERSION": "3.8",
                "PIP_CACHE_DIR": ".cache/pip"
            },
            build_steps=[
                BuildStep(
                    name="Setup Environment",
                    command="pip install -r requirements.txt",
                    timeout=300
                ),
                BuildStep(
                    name="Run Tests",
                    command="python -m pytest tests/ --cov=. --cov-report=xml",
                    timeout=600
                ),
                BuildStep(
                    name="Code Quality Check",
                    command="flake8 . && pylint src/",
                    timeout=300
                ),
                BuildStep(
                    name="Build Package",
                    command="python setup.py sdist bdist_wheel",
                    timeout=300
                )
            ],
            artifacts=[
                "dist/*.whl",
                "coverage.xml",
                "test_report.html"
            ],
            notifications={
                "email": {
                    "on_success": False,
                    "on_failure": True
                },
                "slack": {
                    "webhook_url": "${SLACK_WEBHOOK_URL}",
                    "channel": "#ci-cd"
                }
            }
        )
        
        # LangChain项目模板
        langchain_template = CIConfiguration(
            name="LangChain Project",
            repository="",
            branches=["main", "develop"],
            triggers=[TriggerType.COMMIT, TriggerType.PR],
            environment={
                "PYTHON_VERSION": "3.8",
                "OPENAI_API_KEY": "${OPENAI_API_KEY}",
                "LANGCHAIN_API_KEY": "${LANGCHAIN_API_KEY}"
            },
            build_steps=[
                BuildStep(
                    name="Setup Environment",
                    command="pip install -r requirements.txt",
                    timeout=300
                ),
                BuildStep(
                    name="Install LangChain",
                    command="pip install langchain openai",
                    timeout=300
                ),
                BuildStep(
                    name="Run Unit Tests",
                    command="python -m pytest tests/unit/ --cov=src",
                    timeout=600
                ),
                BuildStep(
                    name="Run Integration Tests",
                    command="python -m pytest tests/integration/",
                    timeout=900
                ),
                BuildStep(
                    name="Test LangChain Components",
                    command="python -m pytest tests/langchain/",
                    timeout=600
                ),
                BuildStep(
                    name="Security Scan",
                    command="bandit -r src/",
                    timeout=300
                )
            ],
            artifacts=[
                "test_results.xml",
                "coverage.xml",
                "security_report.json"
            ],
            notifications={
                "email": {
                    "on_success": False,
                    "on_failure": True
                },
                "slack": {
                    "webhook_url": "${SLACK_WEBHOOK_URL}",
                    "channel": "#langchain-ci"
                }
            }
        )
        
        self.templates["python"] = python_template
        self.templates["langchain"] = langchain_template
    
    def create_configuration(self, name: str, repository: str, 
                             template_name: str = "python") -> CIConfiguration:
        """创建配置"""
        if template_name not in self.templates:
            template_name = "python"
        
        template = self.templates[template_name]
        
        config = CIConfiguration(
            name=name,
            repository=repository,
            branches=template.branches.copy(),
            triggers=template.triggers.copy(),
            environment=template.environment.copy(),
            build_steps=[
                BuildStep(
                    name=step.name,
                    command=step.command,
                    timeout=step.timeout,
                    retry_count=step.retry_count,
                    continue_on_error=step.continue_on_error,
                    working_directory=step.working_directory,
                    environment=step.environment.copy()
                )
                for step in template.build_steps
            ],
            artifacts=template.artifacts.copy(),
            notifications=template.notifications.copy()
        )
        
        self.configurations[name] = config
        print(f"CI配置创建成功: {name}")
        
        return config
    
    def get_configuration(self, name: str) -> Optional[CIConfiguration]:
        """获取配置"""
        return self.configurations.get(name)
    
    def list_templates(self) -> List[str]:
        """列出可用模板"""
        return list(self.templates.keys())

print(f"   ✅ CI核心概念完成")
print(f"      - BuildJob: 构建任务定义")
print(f"      - ContinuousIntegrationServer: CI服务器")
print(f"      - CIConfiguration: CI配置管理")
print(f"      - 支持多种触发类型和构建步骤")

print(f"\n✅ 持续集成（CI）系统完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握CI的核心概念和流程")
print(f"   ✓ 学会构建自动化流水线")
print(f"   ✓ 理解触发机制和构建管理")
print(f"   ✓ 能够建立基础的CI系统")

### 持续部署（CD）系统 [⭐⭐进阶]
**知识点说明**：持续部署系统是自动化交付的核心，为LangChain的AI项目提供可靠的部署能力。

**学习要求**：
- 掌握CD的核心概念和策略
- 学会部署管道和环境管理
- 理解回滚机制和监控告警
- 能够建立完整的CD体系

**案例要求**：
- 实现CD部署管道
- 构建环境管理和配置系统
- 开发部署策略和回滚机制
- 验证点：能独立建立完整的CD体系

In [ ]:
print("\n🚀 持续部署（CD）系统:")
print("=" * 50)

# 2. 持续部署系统
print(f"📝 2. 持续部署系统:")

# 2.1 部署策略
print(f"\n   🎯 2.1 部署策略:")

class DeploymentStrategy(Enum):
    """部署策略"""
    ROLLING = "rolling"  # 滚动部署
    BLUE_GREEN = "blue_green"  # 蓝绿部署
    CANARY = "canary"  # 金丝雀部署
    A_B_TESTING = "ab_testing"  # A/B测试
    RECREATE = "recreate"  # 重建部署

class EnvironmentType(Enum):
    """环境类型"""
    DEVELOPMENT = "development"  # 开发环境
    TESTING = "testing"  # 测试环境
    STAGING = "staging"  # 预发布环境
    PRODUCTION = "production"  # 生产环境

@dataclass
class DeploymentEnvironment:
    """部署环境"""
    name: str
    environment_type: EnvironmentType
    host: str
    port: int
    credentials: Dict[str, str]
    configuration: Dict[str, Any]
    health_check_url: str = ""
    
    @property
    def url(self) -> str:
        return f"http://{self.host}:{self.port}"

@dataclass
class DeploymentStep:
    """部署步骤"""
    name: str
    action: str
    timeout: int = 300
    retry_count: int = 0
    continue_on_error: bool = False
    parameters: Dict[str, Any] = field(default_factory=dict)
    
    def __post_init__(self):
        self.status = BuildStatus.PENDING
        self.start_time: Optional[datetime.datetime] = None
        self.end_time: Optional[datetime.datetime] = None
        self.output: str = ""
        self.error_message: Optional[str] = None

@dataclass
class DeploymentJob:
    """部署任务"""
    id: str
    name: str
    application: str
    version: str
    environment: DeploymentEnvironment
    strategy: DeploymentStrategy
    build_artifacts: List[str]
    steps: List[DeploymentStep]
    triggered_by: str
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    
    def __post_init__(self):
        self.status = BuildStatus.PENDING
        self.start_time: Optional[datetime.datetime] = None
        self.end_time: Optional[datetime.datetime] = None
        self.rollback_available: bool = False
        self.previous_version: Optional[str] = None

@dataclass
class DeploymentResult:
    """部署结果"""
    job: DeploymentJob
    status: BuildStatus
    execution_time: float
    step_results: List[Dict[str, Any]] = field(default_factory=list)
    deployed_version: Optional[str] = None
    rollback_version: Optional[str] = None
    health_check_result: Optional[Dict[str, Any]] = None
    error_message: Optional[str] = None

class DeploymentStrategyManager:
    """部署策略管理器"""
    
    def __init__(self):
        self.strategies = {
            DeploymentStrategy.ROLLING: self._rolling_deployment,
            DeploymentStrategy.BLUE_GREEN: self._blue_green_deployment,
            DeploymentStrategy.CANARY: self._canary_deployment,
            DeploymentStrategy.A_B_TESTING: self._ab_testing_deployment,
            DeploymentStrategy.RECREATE: self._recreate_deployment
        }
    
    def execute_deployment(self, job: DeploymentJob) -> bool:
        """执行部署"""
        strategy_func = self.strategies.get(job.strategy)
        if not strategy_func:
            raise ValueError(f"不支持的部署策略: {job.strategy}")
        
        return strategy_func(job)
    
    def _rolling_deployment(self, job: DeploymentJob) -> bool:
        """滚动部署"""
        print(f"执行滚动部署: {job.application} v{job.version}")
        
        # 模拟滚动部署过程
        steps = [
            "准备新版本",
            "逐步替换实例 (25%)",
            "逐步替换实例 (50%)",
            "逐步替换实例 (75%)",
            "逐步替换实例 (100%)",
            "健康检查",
            "清理旧版本"
        ]
        
        for step in steps:
            print(f"  - {step}")
            time.sleep(0.1)
        
        return True
    
    def _blue_green_deployment(self, job: DeploymentJob) -> bool:
        """蓝绿部署"""
        print(f"执行蓝绿部署: {job.application} v{job.version}")
        
        steps = [
            "准备绿色环境",
            "部署到绿色环境",
            "绿色环境健康检查",
            "切换流量到绿色环境",
            "验证蓝色环境可回滚",
            "清理蓝色环境"
        ]
        
        for step in steps:
            print(f"  - {step}")
            time.sleep(0.1)
        
        return True
    
    def _canary_deployment(self, job: DeploymentJob) -> bool:
        """金丝雀部署"""
        print(f"执行金丝雀部署: {job.application} v{job.version}")
        
        steps = [
            "部署金丝雀版本 (5%流量)",
            "监控金丝雀版本",
            "逐步增加流量 (20%)",
            "监控和验证",
            "逐步增加流量 (50%)",
            "最终验证",
            "全量部署 (100%流量)"
        ]
        
        for step in steps:
            print(f"  - {step}")
            time.sleep(0.1)
        
        return True
    
    def _ab_testing_deployment(self, job: DeploymentJob) -> bool:
        """A/B测试部署"""
        print(f"执行A/B测试部署: {job.application} v{job.version}")
        
        steps = [
            "部署A版本",
            "部署B版本",
            "配置流量分配",
            "启动A/B测试",
            "监控测试结果",
            "分析数据",
            "选择最佳版本"
        ]
        
        for step in steps:
            print(f"  - {step}")
            time.sleep(0.1)
        
        return True
    
    def _recreate_deployment(self, job: DeploymentJob) -> bool:
        """重建部署"""
        print(f"执行重建部署: {job.application} v{job.version}")
        
        steps = [
            "停止所有实例",
            "部署新版本",
            "启动所有实例",
            "健康检查",
            "验证服务可用性"
        ]
        
        for step in steps:
            print(f"  - {step}")
            time.sleep(0.1)
        
        return True

# 2.2 CD部署管道
print(f"\n   🔄 2.2 CD部署管道:")

class ContinuousDeploymentServer:
    """持续部署服务器"""
    
    def __init__(self):
        self.deployment_queue: queue.Queue = queue.Queue()
        self.active_deployments: Dict[str, DeploymentJob] = {}
        self.deployment_history: List[DeploymentResult] = []
        self.environments: Dict[str, DeploymentEnvironment] = {}
        self.strategy_manager = DeploymentStrategyManager()
        self.deployment_workers: List[threading.Thread] = []
        self.max_workers = 2
        self.is_running = False
        self.rollback_manager = RollbackManager()
    
    def start(self):
        """启动CD服务器"""
        self.is_running = True
        
        # 启动部署工作线程
        for i in range(self.max_workers):
            worker = threading.Thread(target=self._deployment_worker, name=f"DeploymentWorker-{i}")
            worker.daemon = True
            worker.start()
            self.deployment_workers.append(worker)
        
        print(f"CD服务器启动，运行 {self.max_workers} 个部署工作线程")
    
    def stop(self):
        """停止CD服务器"""
        self.is_running = False
        print("CD服务器停止")
    
    def add_environment(self, environment: DeploymentEnvironment):
        """添加部署环境"""
        self.environments[environment.name] = environment
        print(f"部署环境添加成功: {environment.name}")
    
    def submit_deployment(self, job: DeploymentJob) -> bool:
        """提交部署任务"""
        try:
            self.deployment_queue.put(job)
            print(f"部署任务提交成功: {job.id} - {job.name}")
            return True
        except Exception as e:
            print(f"部署任务提交失败: {e}")
            return False
    
    def _deployment_worker(self):
        """部署工作线程"""
        while self.is_running:
            try:
                # 获取部署任务（阻塞等待）
                job = self.deployment_queue.get(timeout=1)
                
                # 执行部署
                result = self._execute_deployment(job)
                
                # 记录结果
                self.deployment_history.append(result)
                
                # 清理活跃部署
                if job.id in self.active_deployments:
                    del self.active_deployments[job.id]
                
                print(f"部署完成: {job.id} - {result.status.value}")
                
            except queue.Empty:
                continue
            except Exception as e:
                print(f"部署工作线程错误: {e}")
    
    def _execute_deployment(self, job: DeploymentJob) -> DeploymentResult:
        """执行部署"""
        job.status = BuildStatus.RUNNING
        job.start_time = datetime.datetime.utcnow()
        self.active_deployments[job.id] = job
        
        start_time = time.time()
        step_results = []
        
        try:
            # 部署前检查
            self._pre_deployment_check(job)
            
            # 执行部署步骤
            for step in job.steps:
                step_result = self._execute_deployment_step(step, job)
                step_results.append(step_result)
                
                # 如果步骤失败且不允许继续，停止部署
                if step_result["status"] == BuildStatus.FAILED and not step.continue_on_error:
                    break
            
            # 执行部署策略
            strategy_success = self.strategy_manager.execute_deployment(job)
            
            if strategy_success:
                # 健康检查
                health_result = self._health_check(job)
                
                if health_result["healthy"]:
                    status = BuildStatus.SUCCESS
                    deployed_version = job.version
                    
                    # 记录回滚点
                    self.rollback_manager.create_rollback_point(job)
                    
                else:
                    status = BuildStatus.FAILED
                    deployed_version = None
                    error_message = "健康检查失败"
                    
                    # 自动回滚
                    self._auto_rollback(job)
            else:
                status = BuildStatus.FAILED
                deployed_version = None
                error_message = "部署策略执行失败"
            
        except Exception as e:
            status = BuildStatus.FAILED
            deployed_version = None
            error_message = str(e)
            
            # 自动回滚
            self._auto_rollback(job)
        
        finally:
            execution_time = time.time() - start_time
            job.end_time = datetime.datetime.utcnow()
            job.status = status
        
        return DeploymentResult(
            job=job,
            status=status,
            execution_time=execution_time,
            step_results=step_results,
            deployed_version=deployed_version,
            error_message=error_message
        )
    
    def _pre_deployment_check(self, job: DeploymentJob):
        """部署前检查"""
        print(f"执行部署前检查: {job.application}")
        
        checks = [
            "检查环境可用性",
            "验证构建产物",
            "检查依赖服务",
            "验证配置文件"
        ]
        
        for check in checks:
            print(f"  - {check}")
            time.sleep(0.05)
    
    def _execute_deployment_step(self, step: DeploymentStep, job: DeploymentJob) -> Dict[str, Any]:
        """执行部署步骤"""
        step.status = BuildStatus.RUNNING
        step.start_time = datetime.datetime.utcnow()
        
        start_time = time.time()
        output = ""
        error_message = None
        
        try:
            # 模拟执行部署步骤
            print(f"执行部署步骤: {step.name} - {step.action}")
            
            # 模拟执行时间
            time.sleep(0.1)
            
            # 模拟输出
            output = f"执行动作: {step.action}\n输出: Step completed successfully"
            
            # 模拟随机失败（5%概率）
            import random
            if random.random() < 0.05:
                raise RuntimeError(f"Deployment step failed: {step.name}")
            
            step.status = BuildStatus.SUCCESS
            status = BuildStatus.SUCCESS
            
        except Exception as e:
            step.status = BuildStatus.FAILED
            status = BuildStatus.FAILED
            error_message = str(e)
            output = f"错误: {error_message}"
        
        finally:
            execution_time = time.time() - start_time
            step.end_time = datetime.datetime.utcnow()
            step.output = output
            step.error_message = error_message
        
        return {
            "step_name": step.name,
            "status": status,
            "execution_time": execution_time,
            "output": output,
            "error_message": error_message
        }
    
    def _health_check(self, job: DeploymentJob) -> Dict[str, Any]:
        """健康检查"""
        print(f"执行健康检查: {job.environment.url}")
        
        # 模拟健康检查
        time.sleep(0.2)
        
        import random
        healthy = random.random() < 0.95  # 95%概率健康
        
        result = {
            "healthy": healthy,
            "response_time": random.uniform(50, 200),
            "status_code": 200 if healthy else 500,
            "checks": {
                "database": healthy,
                "redis": healthy,
                "external_apis": healthy
            }
        }
        
        print(f"  健康检查结果: {'通过' if healthy else '失败'}")
        return result
    
    def _auto_rollback(self, job: DeploymentJob):
        """自动回滚"""
        print(f"执行自动回滚: {job.application}")
        
        rollback_success = self.rollback_manager.rollback(job)
        
        if rollback_success:
            print(f"  自动回滚成功")
        else:
            print(f"  自动回滚失败，需要手动干预")
    
    def rollback_deployment(self, job_id: str, triggered_by: str) -> bool:
        """手动回滚部署"""
        # 查找部署历史
        for result in self.deployment_history:
            if result.job.id == job_id and result.status == BuildStatus.SUCCESS:
                print(f"执行手动回滚: {job_id}")
                return self.rollback_manager.rollback(result.job, triggered_by)
        
        print(f"未找到可回滚的部署: {job_id}")
        return False
    
    def get_deployment_status(self, job_id: str) -> Optional[Dict[str, Any]]:
        """获取部署状态"""
        # 检查活跃部署
        if job_id in self.active_deployments:
            job = self.active_deployments[job_id]
            return {
                "job_id": job_id,
                "status": job.status,
                "start_time": job.start_time.isoformat() if job.start_time else None,
                "progress": "running"
            }
        
        # 检查历史部署
        for result in self.deployment_history:
            if result.job.id == job_id:
                return {
                    "job_id": job_id,
                    "status": result.status,
                    "start_time": result.job.start_time.isoformat() if result.job.start_time else None,
                    "end_time": result.job.end_time.isoformat() if result.job.end_time else None,
                    "execution_time": result.execution_time,
                    "deployed_version": result.deployed_version,
                    "progress": "completed"
                }
        
        return None
    
    def get_deployment_statistics(self) -> Dict[str, Any]:
        """获取部署统计"""
        total_deployments = len(self.deployment_history)
        successful_deployments = len([r for r in self.deployment_history if r.status == BuildStatus.SUCCESS])
        failed_deployments = len([r for r in self.deployment_history if r.status == BuildStatus.FAILED])
        
        total_time = sum(r.execution_time for r in self.deployment_history)
        
        return {
            "total_deployments": total_deployments,
            "successful_deployments": successful_deployments,
            "failed_deployments": failed_deployments,
            "success_rate": successful_deployments / total_deployments if total_deployments > 0 else 0,
            "active_deployments": len(self.active_deployments),
            "queued_deployments": self.deployment_queue.qsize(),
            "average_deployment_time": total_time / total_deployments if total_deployments > 0 else 0
        }

class RollbackManager:
    """回滚管理器"""
    
    def __init__(self):
        self.rollback_points: Dict[str, Dict[str, Any]] = {}
    
    def create_rollback_point(self, job: DeploymentJob):
        """创建回滚点"""
        rollback_point = {
            "job_id": job.id,
            "application": job.application,
            "environment": job.environment.name,
            "new_version": job.version,
            "previous_version": job.previous_version,
            "created_at": datetime.datetime.utcnow(),
            "deployed_files": job.build_artifacts.copy(),
            "configuration": job.environment.configuration.copy()
        }
        
        self.rollback_points[job.id] = rollback_point
        print(f"回滚点创建成功: {job.id}")
    
    def rollback(self, job: DeploymentJob, triggered_by: str = "system") -> bool:
        """执行回滚"""
        if job.id not in self.rollback_points:
            print(f"未找到回滚点: {job.id}")
            return False
        
        rollback_point = self.rollback_points[job.id]
        
        print(f"执行回滚: {rollback_point['application']} v{rollback_point['new_version']} -> v{rollback_point['previous_version']}")
        
        # 模拟回滚过程
        steps = [
            "停止当前版本",
            "恢复 previous 版本",
            "恢复配置文件",
            "启动服务",
            "健康检查",
            "验证回滚成功"
        ]
        
        for step in steps:
            print(f"  - {step}")
            time.sleep(0.1)
        
        print(f"回滚完成，触发者: {triggered_by}")
        return True

print(f"   ✅ 持续部署系统完成")
print(f"      - DeploymentStrategyManager: 部署策略管理器")
print(f"      - ContinuousDeploymentServer: CD服务器")
print(f"      - RollbackManager: 回滚管理器")
print(f"      - 支持多种部署策略和自动回滚")

print(f"\n✅ 持续部署（CD）系统完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握CD的核心概念和策略")
print(f"   ✓ 学会部署管道和环境管理")
print(f"   ✓ 理解回滚机制和监控告警")
print(f"   ✓ 能够建立完整的CD体系")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**8.4 持续集成与持续部署 [⭐⭐进阶]**
- ✅ 掌握CI/CD的核心概念和最佳实践
- ✅ 学会构建自动化流水线
- ✅ 理解部署策略和环境管理
- ✅ 能够建立完整的DevOps流程

#### 🎯 核心技能掌握

**持续集成**
- ✅ CI服务器架构设计
- ✅ 构建任务管理和调度
- ✅ 多种触发机制实现
- ✅ 构建环境管理

**持续部署**
- ✅ 部署策略设计和实现
- ✅ 环境管理和配置
- ✅ 自动化部署管道
- ✅ 回滚机制和故障恢复

**DevOps实践**
- ✅ 自动化工具链集成
- ✅ 监控和告警系统
- ✅ 质量门禁和流水线
- ✅ 基础设施即代码

#### 🚀 实践应用能力

**CI/CD流水线**
- ✅ 完整流水线设计和实现
- ✅ 多环境部署管理
- ✅ 自动化测试集成
- ✅ 构建产物管理

**运维自动化**
- ✅ 部署自动化流程
- ✅ 监控和日志管理
- ✅ 故障检测和恢复
- ✅ 性能优化和调优

#### 📊 与LangChain的关联

**AI项目CI/CD**
- ✅ 支持LangChain项目的自动化构建
- ✅ AI模型训练和部署流水线
- ✅ 机器学习管道自动化
- ✅ AI组件的持续集成

**工程化支持**
- ✅ 为LangChain提供完整的CI/CD方案
- ✅ 支持AI项目的持续交付能力
- ✅ 保障AI服务的稳定部署
- ✅ 实现AI系统的自动化运维

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **CI系统设计** - 理解持续集成的核心概念，能够设计和实现完整的CI服务器
2. **CD策略实施** - 掌握多种部署策略，能够建立可靠的持续部署系统
3. **DevOps流程** - 学会自动化工具链集成，能够建立完整的DevOps流程
4. **运维自动化** - 能够实现监控、告警、回滚等运维自动化功能

这些技能为LangChain等AI项目的自动化交付提供了完整的解决方案，确保了项目的持续集成和部署能力。接下来将继续学习性能优化与调试技巧等工程实践技能。